## 1. 코랩에서 자동수집
- 스케줄러 코드 작동이 너무 오래걸려서 되는건지 모르겠음

In [ ]:
#라이브러리 설치
!pip install -q feedparser sqlite-utils schedule

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 6.4 MB/s eta 0:00:00


In [ ]:
import sqlite3

# DB 연결 (없으면 생성됨)
conn = sqlite3.connect("news.db")
cursor = conn.cursor()

# 테이블 생성 (스키마 설계)
cursor.execute("""
CREATE TABLE IF NOT EXISTS articles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    link TEXT UNIQUE,  -- 중복 제거를 위해 UNIQUE
    published TEXT,
    summary TEXT,
    source TEXT,
    category TEXT DEFAULT '미분류'
)
""")
conn.commit()
print("✅ DB 준비 완료")


✅ DB 준비 완료


In [ ]:
import feedparser
from datetime import datetime

def fetch_and_store_news(feed_url, source, max_results=20):
    """
    RSS feed에서 기사 가져와 DB에 저장
    """
    feed = feedparser.parse(feed_url)
    inserted, skipped = 0, 0

    for entry in feed.entries[:max_results]:
        title = entry.title
        link = entry.link
        published = getattr(entry, "published", datetime.now().strftime("%Y-%m-%d"))
        summary = getattr(entry, "summary", "")

        try:
            cursor.execute("""
                INSERT INTO articles (title, link, published, summary, source)
                VALUES (?, ?, ?, ?, ?)
            """, (title, link, published, summary, source))
            conn.commit()
            inserted += 1
        except sqlite3.IntegrityError:
            skipped += 1  # 중복이면 저장 안 함

    print(f"🔹 {source} 수집 완료: {inserted}개 저장, {skipped}개 중복 무시됨")


In [ ]:
# RSS 소스 목록
RSS_FEEDS = {
    "IT동아": "https://it.donga.com/feeds/rss/news/",
    "ZDNet Korea": "https://zdnet.co.kr/news/news_xml.asp",
    "전자신문": "https://rss.etnews.com/Section902.xml"
}

In [ ]:
# 하루 1회 수집
import schedule, time

def job():
    print("⏳ 뉴스 수집 시작")
    for source, url in RSS_FEEDS.items():
        fetch_and_store_news(url, source)
    print("✅ 모든 뉴스 수집 완료!\n")

# 하루 1회 (아침 7시)
schedule.every().day.at("07:00").do(job)

# 테스트용: 1분마다 실행
# schedule.every(1).minutes.do(job)

print("⏳ 스케줄러 실행 중... (강제 종료하려면 Colab 셀 중지)")
while True:
    schedule.run_pending()
    time.sleep(1)

⏳ 스케줄러 실행 중... (강제 종료하려면 Colab 셀 중지)


KeyboardInterrupt: 

In [ ]:
import pandas as pd

df = pd.read_sql("SELECT * FROM articles ORDER BY id DESC LIMIT 10", conn)
df

,id,title,link,published,summary,source,category


## 2. 깃허브 사용
- 개인토큰 사용해서 작동해야 함.
- 팀원 깃허브 아이디 생성하고 수락하면 볼 수 있다고 함

In [ ]:
# ===== 1️⃣ 설치 및 라이브러리 import =====
!pip install feedparser PyGithub

import feedparser
import json
from datetime import datetime
from github import Github

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.5/416.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 52.4 MB/s eta 0:00:00


In [ ]:
# ===== 2️⃣ RSS 뉴스 수집 =====

RSS_URL = 'https://rss.etnews.com/Section901.xml'  # 원하는 RSS 링크로 변경 가능

def fetch_rss_news():
    feed = feedparser.parse(RSS_URL)
    news_list = []
    for entry in feed.entries:
        news_list.append({
            'title': entry.title,
            'summary': entry.summary,
            'link': entry.link,
            'published': entry.published if 'published' in entry else ''
        })
    return news_list

In [ ]:
# ===== 3️⃣ JSON으로 저장 =====
def save_news_json(news_list):
    filename = 'news_data.json'
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(news_list, f, ensure_ascii=False, indent=4)
    print(f"{filename} 저장 완료!")

In [ ]:
# ===== 4️⃣ GitHub 업로드 함수 =====
GITHUB_TOKEN = 'ghp_xx'
GITHUB_REPO = 'jaehui526/news-collector'

def upload_to_github(filename):
    g = Github(GITHUB_TOKEN)
    repo = g.get_repo(GITHUB_REPO)

    with open(filename, 'r', encoding='utf-8') as f:
        content = f.read()

    try:
        contents = repo.get_contents(filename)
        repo.update_file(contents.path, f"Update news {datetime.now().strftime('%Y-%m-%d %H:%M')}", content, contents.sha)
        print("GitHub 파일 업데이트 완료!")
    except:
        repo.create_file(filename, f"Add news {datetime.now().strftime('%Y-%m-%d %H:%M')}", content)
        print("GitHub 파일 신규 업로드 완료!")

In [ ]:
# ===== 5️⃣ 전체 실행 =====
if __name__ == "__main__":
    news = fetch_rss_news()
    if news:
        save_news_json(news)
        upload_to_github('news_data.json')

news_data.json 저장 완료!
GitHub 파일 신규 업로드 완료!
